# Problema de negócio

Empresa **STAR JEANS**

Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários
negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um
modelo de negócio do tipo E-commerce.

A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso
o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação
baixo e escalar a medida que forem conseguindo clientes.

Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência
nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e
o material para a fabricação de cada peça.

Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes
perguntas: 

   1. Qual o melhor preço de venda para as calças? 
   2. Quantos tipos de calças e suas cores para o produto inicial? 
   3. Quais as matérias-prima necessárias para confeccionar as calças?

As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

# Planejamento para solução

 ## Problema de Negócio
Qual o melhor preço de venda para calças?

## Saída do Projeto ( Produto final )
1. A resposta para a pergunta
    - Mediana dos preços dos concorrentes
 
 
2. Formato da entrega
    - Tabela ou Gráfico
 
 
3. Local da entrega
    - App Streamlit


## Processo ( Passo a Passo )

1. Passo a passso para construir o cálculo da mediana ou média
    - Realizar o calculo da mediana sobre o produto, tipo e cor


2. Definir o formato da entrega ( Visualização, Tabela, Frase )
    - Gráfico de barras com a mediana dos preço dos produtos, por tipo e cor dos últimos 30 dias.
    - Tabela com as seguintes colunas: id | product_name | product_type | product_color | product_price
    - Definição do schema: Colunas e seu tipo
    - Definição a infraestrutura de armazenamento ( SQLITE3 )
    - Design do ETL ( Scripts de Extração, Transformação e Carga )
    - Planejamento de Agendamento dos scripts ( dependencias entre os scripts )
    - Fazer as visualizações
    - Entrega do produto final


3. Decidir o local de entrega ( PowerBi, Telegram, Email, Streamlit, Intranet )
    - App com Streamlit


## Entrada ( Fonte de dados )

### 1. Fonte de dados
- Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
- Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

### 2. Ferramentas
- Python 3.8.0
- Bibliotecas de Webscrapping ( BS4, Selenium )
- PyCharm
- Jupyter Notebook ( Analise e prototipagens )
- Crontjob, Airflow
- Streamlit

# 1.0. Web scraping

In [1]:
import requests
import pandas as pd

from bs4      import BeautifulSoup
from datetime import datetime

## 1.1. Número Max de páginas

In [2]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, 'html.parser' )

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item = int( total_item )

# round to the top value
page_number = np.ceil( total_item/36 )

url02 = url + '?page-size=' + str( int( page_number*36 ) )
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=72'

## 1.2. Coleta de dados

In [4]:
url = url02

custom_header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
page = requests.get( url, headers=custom_header )

soup = BeautifulSoup( page.text, 'html.parser' )

products = soup.find( 'ul', class_='products-listing small' )
products_list = products.find_all( 'article', class_='hm-product-item')

#id
product_id = [p.get( 'data-articlecode' ) for p in products_list]

#category
product_cat = [p.get( 'data-category' ) for p in products_list]

# name
product_name = [p.find( 'a', class_='link' ).get_text() for p in products_list]

# price
product_price = [p.find( 'span', class_='price regular' ).get_text() for p in products_list]

# dataframe from the products showed in main page
data = pd.DataFrame( [product_id, product_cat, product_name, product_price] ).T
data.columns = ['id', 'category','name','price']

# creating two differents columns from the ID wich represents a compound information 
# about color and style
data['style_id'] = data['id'].apply( lambda x: x[:-3] )
data['color_id'] = data['id'].apply( lambda x: x[-3:] )

data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

# general data
data

,id,category,name,price,style_id,color_id,scrapy_datetime
0,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-24 14:12:42
1,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,043,2021-12-24 14:12:42
2,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,022,2021-12-24 14:12:42
3,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,051,2021-12-24 14:12:42
4,0690449056,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,056,2021-12-24 14:12:42
...,...,...,...,...,...,...,...
61,0974202002,men_jeans_loose,Regular Denim Joggers,$ 29.99,0974202,002,2021-12-24 14:12:42
62,0875105009,men_jeans_relaxed,Relaxed Jeans,$ 29.99,0875105,009,2021-12-24 14:12:42
63,1014096001,men_jeans_loose,Cotton Denim Loose Jeans,$ 49.99,1014096,001,2021-12-24 14:12:42
64,0875105001,men_jeans_relaxed,Relaxed Jeans,$ 29.99,0875105,001,2021-12-24 14:12:42


## 1.3. Color and Composition

In [9]:
custom_header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
df_details = pd.DataFrame()

# unique columns for all products 
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )

# Using just the id from the data to go through each product and catch each feature inside the page of each product
for i in range( len(data) ):

    # Api Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i,'id'] + '.html'
    
    page = requests.get( url, headers=custom_header )
    
    # Beautifulsoup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    # color name and product id
    product_list = soup.find_all( 'a', class_='filter-option miniature' )
    
    color_name = [p.get( 'data-color' ) for p in product_list]
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    
    # color and id data frame
    df_color = pd.DataFrame( [product_id,color_name] ).T
    df_color.columns = ['product_id','color_name']
    
    # generate style_id e color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )
    
    # composition 
    product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
    
    composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]
    
    # rename dataframe
    df_aux = pd.DataFrame( composition ).T
    df_aux.columns = df_aux.iloc[0]
    
    df_aux = df_aux.iloc[1:].fillna( method='ffill' ).reset_index(drop=True)
    
    # garantee the same number of columns
    df_aux = pd.concat( [df_pattern, df_aux], axis=0 )
    
    #generate style id + color id
    df_aux['style_id'] = df_aux['Art. No.'].apply( lambda x: x[:-3] )
    df_aux['color_id'] = df_aux['Art. No.'].apply( lambda x: x[-3:] )
    
    # just to know how many features has in the diferents products
    aux = aux + df_aux.columns.tolist()
    
    # merge
    df_sku = pd.merge( df_color, df_aux[['style_id','Fit','Composition','Size','More sustainable materials', 'Product safety']], how='left', on='style_id' )
    
    df_details = pd.concat( [df_details, df_sku], axis=0 )
    
df_details = df_details.reset_index(drop=True)
data_raw = pd.merge( data, df_details[['style_id','color_name','Fit','Composition','Size', 'Product safety']], how='left', on='style_id' )

In [13]:
data_raw.head()

,id,category,name,price,style_id,color_id,scrapy_datetime,color_name,Fit,Composition,Size,Product safety
0,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-24 14:12:42,Light denim blue/trashed,Skinny fit,"Cotton 98%, Spandex 2%","The model is 188cm/6'2"" and wears a size 31/30",NaN
1,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-24 14:12:42,Denim blue,Skinny fit,"Cotton 98%, Spandex 2%","The model is 188cm/6'2"" and wears a size 31/30",NaN
2,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-24 14:12:42,Black/washed,Skinny fit,"Cotton 98%, Spandex 2%","The model is 188cm/6'2"" and wears a size 31/30",NaN
3,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-24 14:12:42,Light denim blue,Skinny fit,"Cotton 98%, Spandex 2%","The model is 188cm/6'2"" and wears a size 31/30",NaN
4,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-24 14:12:42,Black washed out,Skinny fit,"Cotton 98%, Spandex 2%","The model is 188cm/6'2"" and wears a size 31/30",NaN


In [28]:
# difinition to var "cols" above
set(aux)

{'Art. No.',
 'Composition',
 'Fit',
 'More sustainable materials',
 'Product safety',
 'Size',
 'color_id',
 'style_id'}